In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [2]:
from fastai.dataloader import *
from fastai.dataset import *
from fastai.transforms import *
from fastai.models import *
from fastai.conv_learner import *

In [3]:
DIR = Path('data/imagenet/')
TRAIN_CSV='train.csv'

In [4]:
from pathlib import Path

## Load ImageNet sample and format data

In [ ]:
!wget --header="Host: files.fast.ai" --header="User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/65.0.3325.146 Safari/537.36" --header="Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8" --header="Accept-Language: en-GB,en-US;q=0.9,en;q=0.8" "http://files.fast.ai/data/imagenet-sample-train.tar.gz" -O "imagenet-sample-train.tar.gz" -c

In [ ]:
!mkdir data/imagenet

In [ ]:
!tar -xzf imagenet-sample-train.tar.gz -C data/imagenet

In [ ]:
ls data/imagenet/train/n01558993

In [ ]:
TRAIN = Path('data/imagenet/train')

In [ ]:
CATEGORIES = list(Path(TRAIN).iterdir())

In [ ]:
cats_to_files = {cat : list(cat.iterdir()) for cat in CATEGORIES}

In [ ]:

files = []
for k,v in cats_to_files.items():
    for file in v:
        files.append(file)

In [ ]:

!mkdir data/imagenet/train1

In [ ]:
import shutil  
for file in files:
    shutil.move(str(file), 'data/imagenet/train1')

In [ ]:
ALL_FILES = Path('data/imagenet/train1')

In [ ]:
afiles = list(ALL_FILES.iterdir())

In [ ]:

import pandas as pd

In [ ]:
IMAGE='image'
CATEGORY='category'
df = pd.DataFrame(afiles, columns=[IMAGE])

In [ ]:

df[IMAGE] = df[IMAGE].apply(lambda x: str(x)[str(x).rfind('/')+1:])

In [ ]:
df['category'] = df[IMAGE].apply(lambda x: x[:x.find('_')])

In [ ]:
df.to_csv(DIR/TRAIN_CSV, index=False)

### Training model

In [5]:
!ls {DIR}

models	tmp  train  train1  train.csv


In [6]:
arch = resnet34
tfms = tfms_from_model(arch, 256, aug_tfms=transforms_side_on)
bs = 128

In [7]:
data = ImageClassifierData.from_csv(DIR, 'train1', DIR/TRAIN_CSV, tfms=tfms, bs=bs)

In [8]:
m = ConvnetBuilder(resnet34, data.c, data.is_multi, data.is_reg)
# models = ConvnetBuilder(resnet34, data.c, data.is_multi, data.is_reg)
# m.model = network_to_half(m.model)
learner = ConvLearner(data, m)

In [9]:
%time learner.fit(0.5,1,cycle_len=1)

HBox(children=(IntProgress(value=0, description='Epoch', max=1), HTML(value='')))

epoch      trn_loss   val_loss   accuracy                   
    0      2.918728   1.625027   0.620684  

CPU times: user 5min 27s, sys: 2min 46s, total: 8min 13s
Wall time: 1min 10s


[1.6250268, 0.6206835527573863]

### Training model 16

In [10]:
import torch
import torch.nn as nn


class tofp16(nn.Module):
    def __init__(self):
        super(tofp16, self).__init__()

    def forward(self, input):
        return input.half()


def copy_in_params(net, params):
    net_params = list(net.parameters())
    for i in range(len(params)):
        net_params[i].data.copy_(params[i].data)


def set_grad(params, params_with_grad):

    for param, param_w_grad in zip(params, params_with_grad):
        if param.grad is None:
            param.grad = torch.nn.Parameter(param.data.new().resize_(*param.data.size()))
        param.grad.data.copy_(param_w_grad.grad.data)


def BN_convert_float(module):
    '''
    BatchNorm layers to have parameters in single precision.
    Find all layers and convert them back to float. This can't
    be done with built in .apply as that function will apply
    fn to all modules, parameters, and buffers. Thus we wouldn't
    be able to guard the float conversion based on the module type.
    '''
    if isinstance(module, torch.nn.modules.batchnorm._BatchNorm):
        module.float()
    for child in module.children():
        BN_convert_float(child)
    return module


def network_to_half(network):
    return nn.Sequential(tofp16(), BN_convert_float(network.half()))

In [ ]:
arch = resnet34
tfms = tfms_from_model(arch, 256, aug_tfms=transforms_side_on)
bs = 128

In [ ]:
data = ImageClassifierData.from_csv(DIR, 'train1', DIR/TRAIN_CSV, tfms=tfms, bs=bs)

In [11]:
m16 = ConvnetBuilder(resnet34, data.c, data.is_multi, data.is_reg)
m16.model = network_to_half(m16.model)
learner16 = ConvLearner(data, m16)

In [12]:
learner16.model

Sequential(
  (0): tofp16(
  )
  (1): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
    (2): ReLU(inplace)
    (3): MaxPool2d(kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), dilation=(1, 1), ceil_mode=False)
    (4): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
        (relu): ReLU(inplace)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
        (relu): ReLU(inplace)
        (conv2): Conv2d(64, 64, kernel_size=(3,

In [14]:
%time learner16.fit(0.5,1,cycle_len=1)

HBox(children=(IntProgress(value=0, description='Epoch', max=1), HTML(value='')))

epoch      trn_loss   val_loss   accuracy                   
    0      3.002965   1.667827   0.600142  

CPU times: user 5min 20s, sys: 2min 37s, total: 7min 58s
Wall time: 1min 6s


[1.667827, 0.6001415579549728]

## Results

No speedup here. I've confirmed